In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd

import csv
import os
path = '/content/drive/My Drive/Colab Notebooks/'

In [4]:
! pip install torchtext==0.6.0

     |████████████████████████████████| 64 kB 1.5 MB/s 
     |████████████████████████████████| 1.2 MB 11.1 MB/s 
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.11.0
    Uninstalling torchtext-0.11.0:
      Successfully uninstalled torchtext-0.11.0


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.data import Field, BucketIterator
from torchtext.datasets import SequenceTaggingDataset

import spacy
import numpy as np

import time
import random

import gensim
from collections import Counter

In [6]:
cd /content/drive/My Drive/Colab Notebooks

/content/drive/My Drive/Colab Notebooks


In [7]:
!gunzip glove.6B.100d.gz

gzip: glove.6B.100d.gz: No such file or directory


In [8]:
import os
os.chdir("/content/drive/My Drive/Colab Notebooks")
!ls

544_hw4_task1.ipynb  dev	    dev2.out	   test       train
544_hw4_task2.ipynb  dev1_eval.txt  dev.csv	   test1.out  train.tsv
blstm1.pt	     dev1.out	    dev.tsv	   test2.out
blstm2.pt	     dev2_eval.txt  glove.6B.100d  test.csv


In [9]:
def get_coefs(word,*arr): 
  return word, np.asarray(arr, dtype='float32')

In [10]:
embeddings_dictionary = dict()
embedding_dim = 100

# Load GloVe 100D embeddings
with open('/content/drive/My Drive/Colab Notebooks/glove.6B.100d','r',encoding='utf-8') as fp:
    for line in fp.readlines():
        records = line.split()
        word = records[0]
        vector_dimensions = np.asarray(records[1:], dtype='float32')
        embeddings_dictionary [word] = vector_dimensions

In [11]:
#change word to lowercase
word_field = Field(lower=True)
tag_field = Field(unk_token=None)
train_dataset, val_dataset = SequenceTaggingDataset.splits(path=path, train="train.tsv", validation="dev.tsv",
          fields=(("word", word_field), ("tag", tag_field)))

print('This train dataset have %d sentences.'%len(train_dataset))
print('This validation dataset have %d sentences.'%len(val_dataset))

This train dataset have 14041 sentences.
This validation dataset have 3250 sentences.


In [12]:
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec


glove_file = datapath('/content/drive/My Drive/Colab Notebooks/glove.6B.100d')
word2vec_glove_file = get_tmpfile("glove.6B.100d.word2vec")
glove2word2vec(glove_file, word2vec_glove_file)

(400000, 100)

In [13]:
# use glove word2vec model from gensim library
glove_model = KeyedVectors.load_word2vec_format(word2vec_glove_file)

In [14]:
from torchtext.vocab import Vocab

In [15]:
embedding_dim = 100
# initiate vocab by building custom Counter based on word2vec model
word_freq = {word: glove_model.vocab[word].count for word in glove_model.vocab}
word_counter = Counter(word_freq)
word_field.vocab = Vocab(word_counter, min_freq=3)
# mapping each vector/embedding from word2vec model to word_field vocabs
vectors = []
for word, idx in word_field.vocab.stoi.items():
  if word in glove_model.vocab.keys():
    vectors.append(torch.as_tensor(glove_model[word].tolist()))
  else:
    vectors.append(torch.zeros(embedding_dim))
word_field.vocab.set_vectors(stoi=word_field.vocab.stoi, vectors=vectors, dim=embedding_dim)

In [16]:
batch_size=16

In [17]:
tag_field.build_vocab(train_dataset.tag)
# create iterator for batch input
train_iter, val_iter = BucketIterator.splits(datasets=(train_dataset, val_dataset), batch_size=batch_size)
word_pad_idx = word_field.vocab.stoi[word_field.pad_token]
tag_pad_idx = tag_field.vocab.stoi[tag_field.pad_token]

In [18]:
shape = glove_model.vectors.shape
shape

(400000, 100)

In [19]:
print(tag_field.vocab.stoi)

defaultdict(None, {'<pad>': 0, 'O': 1, 'B-LOC': 2, 'B-PER': 3, 'B-ORG': 4, 'I-PER': 5, 'I-ORG': 6, 'B-MISC': 7, 'I-LOC': 8, 'I-MISC': 9})


In [20]:
import torch.nn.functional as F

In [21]:
class BiLSTM2(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, input_dim, emb_dropout,
                 lstm_dropout, fc_dropout, output_dim, word_pad_idx):
        super(BiLSTM2, self).__init__()

        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.input_dim = input_dim
        self.emb_dropout = emb_dropout
        self.fc_dropout = fc_dropout
        self.output_dim = output_dim
        self.word_pad_idx = word_pad_idx
        
        self.embeddings = nn.Embedding(num_embeddings=input_dim, embedding_dim=embedding_dim,
                                       padding_idx=word_pad_idx)
        # dropout before bilstm layer
        self.emb_dropout = nn.Dropout(emb_dropout)
        
        self.lstm = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim,
                            num_layers=1, bidirectional=True, dropout=lstm_dropout)
        # dropout after bilstm layer
        self.fc_dropout = nn.Dropout(fc_dropout)
        self.fc = nn.Linear(hidden_dim*2, output_dim)
        #self.fc_dropout = nn.Dropout(fc_dropout)

    def forward(self, sentence):
        embedding_out = self.embeddings(sentence)
        lstm_out, _ = self.lstm(embedding_out)
        tag_space = self.fc(self.fc_dropout(lstm_out))
        tag_score = F.elu(tag_space)
        return tag_score
    
    def init_weight(self):
        for name, param in self.named_parameters():
            nn.init.normal_(param.data, mean=0, std=0.1)

    def init_embeddings(self, word_pad_idx, pretrained=None, freeze=True):
        self.embeddings.weight.data[word_pad_idx] = torch.zeros(self.embedding_dim)
        if pretrained is not None:
          # use built in function: from pretrained
          self.embeddings = nn.Embedding.from_pretrained(torch.FloatTensor(pretrained),
                                                          padding_idx=word_pad_idx)
            
    def count_parameters(self):
        return sum(p.numel() for p in self.parameters() if p.requires_grad)

In [22]:
bilstm2 = BiLSTM2(embedding_dim=100, hidden_dim=256, input_dim =len(word_field.vocab), emb_dropout=0.3, lstm_dropout=0.33, fc_dropout=0.25, output_dim=128, word_pad_idx = word_pad_idx)
bilstm2.init_weight()
bilstm2.init_embeddings(word_pad_idx=word_pad_idx, pretrained=word_field.vocab.vectors)
print(f"The model has {bilstm2.count_parameters():,} trainable parameters.")
print(bilstm2)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.33 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


The model has 798,848 trainable parameters.
BiLSTM2(
  (embeddings): Embedding(400000, 100, padding_idx=1)
  (emb_dropout): Dropout(p=0.3, inplace=False)
  (lstm): LSTM(100, 256, dropout=0.33, bidirectional=True)
  (fc_dropout): Dropout(p=0.25, inplace=False)
  (fc): Linear(in_features=512, out_features=128, bias=True)
)


In [23]:
optimizer = optim.SGD(bilstm2.parameters(), lr=0.01, momentum=0.9)
n_epochs = 10

In [24]:
class NER(object):

  def __init__(self, model, tag_pad_idx, train_iter, val_iter, word_field, tag_field, optimizer, loss_fn):
    self.model = model
    self.optimizer = optimizer

    self.tag_pad_idx = tag_pad_idx
    self.train_iter = train_iter
    self.val_iter = val_iter
    self.word_field = word_field
    self.tag_field=tag_field

    self.loss_fn = loss_fn(ignore_index=tag_pad_idx)

  @staticmethod
  def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

  def accuracy(self, preds, y):
    max_preds = preds.argmax(dim=1, keepdim=True)  # get the index of the max probability
    non_pad_elements = (y!=tag_pad_idx).nonzero()  # prepare masking for paddings
    correct = max_preds[non_pad_elements].squeeze(1)
    correct_sum = correct.eq(y[non_pad_elements]).sum()
    #correct_sum / torch.FloatTensor([y[non_pad_elements].shape[0]])
    
    return correct_sum / torch.FloatTensor([y[non_pad_elements].shape[0]])


  def epoch(self):
      epoch_loss = 0
      epoch_acc = 0
      self.model.train()
      for batch in train_iter:
        # text :sent len, batch size
        text = batch.word
        # tags :sent len, batch size
        true_tags = batch.tag
        self.optimizer.zero_grad()
        pred_tags = self.model(text)
        # to calculate the loss and accuracy, we flatten both prediction and true tags
        # flatten pred_tags to [sent len, batch size, output dim]
        pred_tags = pred_tags.view(-1, pred_tags.shape[-1])
        # flatten true_tags to [sent len * batch size]
        true_tags = true_tags.view(-1)
        #confusion_matrix = confusion_matrix(true_tags, pred_tags)
        batch_loss = self.loss_fn(pred_tags, true_tags)
        batch_acc = self.accuracy(pred_tags, true_tags)
        batch_loss.backward()
        self.optimizer.step()
        epoch_loss += batch_loss.item()
        epoch_acc += batch_acc.item()
      return epoch_loss / len(train_iter), epoch_acc / len(train_iter)

  def evaluate(self, iterator):
      epoch_loss = 0
      epoch_acc = 0
      self.model.eval()
      with torch.no_grad():
          for batch in iterator:
              text = batch.word
              true_tags = batch.tag
              pred_tags = self.model(text)
              pred_tags = pred_tags.view(-1, pred_tags.shape[-1])
              true_tags = true_tags.view(-1)
              #confu_matrix = confusion_matrix(true_tags, pred_tags)
              batch_loss = self.loss_fn(pred_tags, true_tags)
              batch_acc = self.accuracy(pred_tags, true_tags)
              epoch_loss += batch_loss.item()
              epoch_acc += batch_acc.item()
      return epoch_loss / len(iterator), epoch_acc / len(iterator)

  # main training sequence
  def train(self, n_epochs):
    for epoch in range(n_epochs):
        start_time = time.time()
        train_loss, train_acc = self.epoch()
        end_time = time.time()
        epoch_mins, epoch_secs = NER.epoch_time(start_time, end_time)
        print(f"Epoch: {epoch + 1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s")
        print(f"\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc * 100:.2f}%")
        val_loss, val_acc = self.evaluate(val_iter)
        print(f"\tValidation Loss: {val_loss:.3f} | Validation Acc: {val_acc * 100:.2f}%")
    print(f"\tValidation Loss: {val_loss:.3f} | Validation Acc: {val_acc * 100:.2f}%")
    #print(f"\tVallidation Confusion Metrix: {confu_matrix:.3f}%")

  def infer(self, sentence, true_tags=None, sentence_idxs=None):
    self.model.eval()
    # tokenize sentence
    tokens = [token for token in sentence]
    # transform to indices based on corpus vocab
    numericalized_tokens = [self.word_field.vocab.stoi[t] for t in tokens]
    # begin prediction
    token_tensor = torch.LongTensor(numericalized_tokens)
    token_tensor = token_tensor.unsqueeze(-1)
    predictions = self.model(token_tensor)
    # convert results to tags
    top_predictions = predictions.argmax(-1)
    predicted_tags = [tag_field.vocab.itos[t.item()] for t in top_predictions]
    # print inferred tags
    #max_len_token = len(tokens) + len("word")
    #max_len_tag = len(predicted_tags) + len("pred")
    #print('sentence_idx.       word.      gold tag.       pred tag" )
    #pred_outfile = open('/content/drive/My Drive/Colab Notebooks/dev1.out', 'w')
    #for i, token in enumerate(tokens):
      #print(predicted_tags[i])
      #pred_outfile.write(str(sentence_idxs[i]) + '\t' + token + '\t' + true_tags[i] + '\t' + predicted_tags[i] + "\n")
    #pred_outfile.close()
    return predicted_tags

In [25]:
ner = NER(
  model=bilstm2,
  optimizer=optimizer,
  tag_pad_idx=tag_pad_idx, train_iter=train_iter, val_iter=val_iter,
   word_field=word_field, tag_field =tag_field, loss_fn=nn.CrossEntropyLoss
)

In [26]:
!pip3.7 install torch torchvision

In [27]:
ner.train(n_epochs)

Epoch: 01 | Epoch Time: 0m 47s
	Train Loss: 0.506 | Train Acc: 87.06%
	Validation Loss: 0.527 | Validation Acc: 89.02%
Epoch: 02 | Epoch Time: 0m 45s
	Train Loss: 0.293 | Train Acc: 91.44%
	Validation Loss: 0.413 | Validation Acc: 91.57%
Epoch: 03 | Epoch Time: 0m 46s
	Train Loss: 0.243 | Train Acc: 92.86%
	Validation Loss: 0.359 | Validation Acc: 92.67%
Epoch: 04 | Epoch Time: 0m 48s
	Train Loss: 0.216 | Train Acc: 93.70%
	Validation Loss: 0.318 | Validation Acc: 93.33%
Epoch: 05 | Epoch Time: 0m 51s
	Train Loss: 0.197 | Train Acc: 94.32%
	Validation Loss: 0.287 | Validation Acc: 93.83%
Epoch: 06 | Epoch Time: 0m 48s
	Train Loss: 0.180 | Train Acc: 94.77%
	Validation Loss: 0.269 | Validation Acc: 94.41%
Epoch: 07 | Epoch Time: 0m 45s
	Train Loss: 0.167 | Train Acc: 95.11%
	Validation Loss: 0.249 | Validation Acc: 94.63%
Epoch: 08 | Epoch Time: 0m 45s
	Train Loss: 0.156 | Train Acc: 95.42%
	Validation Loss: 0.236 | Validation Acc: 94.92%
Epoch: 09 | Epoch Time: 0m 45s
	Train Loss: 0.14

In [28]:
# save model
torch.save(bilstm2, '/content/drive/My Drive/Colab Notebooks/blstm2.pt')

In [29]:
#dev
dev_df = pd.read_csv('dev.csv')
dev_df.head()

,sentence_idx,word,NER_tag
0,1,CRICKET,O
1,2,-,O
2,3,LEICESTERSHIRE,B-ORG
3,4,TAKE,O
4,5,OVER,O


In [30]:
# get prediction for dev file
dev_sentence = dev_df['word'].tolist()
dev_sentence_idx = dev_df['sentence_idx'].tolist()
dev_true_tags = dev_df['NER_tag'].tolist

In [31]:
dev_pred_tags = ner.infer(sentence=dev_sentence, true_tags=dev_true_tags, sentence_idxs=dev_sentence_idx)

In [32]:
dev_pred_tags_df = pd.DataFrame({'pred_tag': dev_pred_tags})
new_dev_df = pd.concat([dev_df,dev_pred_tags_df], axis=1)
new_dev_df

,sentence_idx,word,NER_tag,pred_tag
0,1,CRICKET,O,O
1,2,-,O,O
2,3,LEICESTERSHIRE,B-ORG,O
3,4,TAKE,O,O
4,5,OVER,O,O
...,...,...,...,...
51573,1,--,O,O
51574,2,Dhaka,B-ORG,O
51575,3,Newsroom,I-ORG,O
51576,4,880-2-506363,O,O


In [33]:
new_dev_df.to_csv('/content/drive/My Drive/Colab Notebooks/dev2_eval.txt', sep = ' ', index=False, header=False)

In [34]:
out_dev_df = new_dev_df.drop(columns='NER_tag')
out_dev_df.head()

,sentence_idx,word,pred_tag
0,1,CRICKET,O
1,2,-,O
2,3,LEICESTERSHIRE,O
3,4,TAKE,O
4,5,OVER,O


In [35]:
out_dev_df.to_csv('/content/drive/My Drive/Colab Notebooks/dev2.out', sep = ' ', index=False, header=False)

In [36]:
# get prediction for text file
test_df = pd.read_csv('test.csv')
test_df.head()


,sentence_idx,word
0,1,SOCCER
1,2,-
2,3,JAPAN
3,4,GET
4,5,LUCKY


In [37]:
test_sentence = test_df['word'].tolist()
test_sentence_idx = test_df['sentence_idx'].tolist()

In [38]:
test_pred_tags = ner.infer(sentence=test_sentence, sentence_idxs=test_sentence_idx)

In [39]:
test_pred_tags_df = pd.DataFrame({'pred_tag': test_pred_tags})
new_test_df = pd.concat([test_df,test_pred_tags_df], axis=1)
new_test_df.head()

,sentence_idx,word,pred_tag
0,1,SOCCER,O
1,2,-,O
2,3,JAPAN,O
3,4,GET,O
4,5,LUCKY,O


In [40]:
new_test_df.to_csv('/content/drive/My Drive/Colab Notebooks/test2.out', sep = ' ', index=False, header=False)